# GAIA Data Homework

In [2]:
import numpy as np
from astropy.table import QTable, Column
from astroquery.gaia import Gaia

### Get a list of ALL the Gaia objects you would see straight above your head (Zenith) at midnight tonight (Jan 17th).

### You will need information from your [Astro Info Sheet](./Info/AstroInfo.pdf).

### At midnight tonight:

  - The Local Sidereal Time (LST) for Seattle (this room) at local midnight is 7h 59m 6s  (119.8 deg).

### You want ALL objects with 

- `ra` and `dec` within 1 degree of the Zenith
- `phot_g_mean_mag` < 10
- `parallax` > 0
- You also want the `source_id` and color index (`bp_rp`) for all objects
- Make sure you adjust your `SELECT` to include all objects

In [3]:
query = """
SELECT TOP 100
source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax
FROM gaiadr3.gaia_source_lite
WHERE
    DISTANCE( POINT(119.8, 47.0), POINT(ra, dec) ) < 1.0
    AND phot_g_mean_mag < 10.0
    AND parallax > 0.0
    AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC
"""

In [4]:
job_query = Gaia.launch_job(query)
print(job_query)

<Table length=36>
      name       dtype  unit                            description                            
--------------- ------- ---- ------------------------------------------------------------------
      source_id   int64      Unique source identifier (unique within a particular Data Release)
             ra float64  deg                                                    Right ascension
            dec float64  deg                                                        Declination
phot_g_mean_mag float32  mag                                              G-band mean magnitude
          bp_rp float32  mag                                                     BP - RP colour
       parallax float64  mas                                                           Parallax
Jobid: None
Phase: COMPLETED
Owner: None
Output file: 1705712213579O-result.vot.gz
Results: None


In [5]:
star_table = job_query.get_results()
star_table

source_id,ra,dec,phot_g_mean_mag,bp_rp,parallax
,deg,deg,mag,mag,mas
int64,float64,float64,float32,float32,float64
932851782702640128,121.1837299677284,47.28846093314972,9.224431,0.59405327,3.9774703932418434
932376931117076608,119.35794145891437,46.3384471259334,9.816679,0.67705536,6.234940232318302
932471420398832000,119.9196815899295,46.61987911083302,8.016949,0.6863818,6.957786644888596
932931359856644224,120.55034374781623,47.58860439361576,9.678056,0.692132,6.90085897359672
933103811383007744,119.9957105955765,47.78505238304345,8.517405,0.7436218,9.864936527454228
932764715125240576,118.8294649241317,47.42740866769713,9.089646,0.7580681,2.2550859647279298
932778562099849856,119.03617666610461,47.55034822924161,9.888519,0.895524,10.587155139199597
932384043584096384,119.5069805080389,46.43745623074234,8.470822,1.1134033,2.4361439091131536


## Print the table row that contains the brightest object

In [6]:
star_table[star_table['phot_g_mean_mag'] == star_table['phot_g_mean_mag'].min()]

source_id,ra,dec,phot_g_mean_mag,bp_rp,parallax
,deg,deg,mag,mag,mas
int64,float64,float64,float32,float32,float64
932769353689866752,118.6776000975415,47.564599416037275,4.941986,1.6506271,4.552206476653628


## Add a column `distance` to your table that has the distance (in parsecs) for all of the objects

In [7]:
def dpc(p):
    return (1 / (p/1000))

In [8]:
distance_values = [dpc(p = parallax_value) for parallax_value in star_table['parallax']]
distance_column = Column(distance_values, name='distance', dtype='float')
star_table.add_column(distance_column)

star_table

source_id,ra,dec,phot_g_mean_mag,bp_rp,parallax,distance
,deg,deg,mag,mag,mas,
int64,float64,float64,float32,float32,float64,float64
932851782702640128,121.1837299677284,47.28846093314972,9.224431,0.59405327,3.9774703932418434,251.41607633311597
932376931117076608,119.35794145891437,46.3384471259334,9.816679,0.67705536,6.234940232318302,160.38646125532716
932471420398832000,119.9196815899295,46.61987911083302,8.016949,0.6863818,6.957786644888596,143.7238666601872
932931359856644224,120.55034374781623,47.58860439361576,9.678056,0.692132,6.90085897359672,144.90949660413088
933103811383007744,119.9957105955765,47.78505238304345,8.517405,0.7436218,9.864936527454228,101.3691266250917
932764715125240576,118.8294649241317,47.42740866769713,9.089646,0.7580681,2.2550859647279298,443.4420752206878
932778562099849856,119.03617666610461,47.55034822924161,9.888519,0.895524,10.587155139199597,94.45408014259073
932384043584096384,119.5069805080389,46.43745623074234,8.470822,1.1134033,2.4361439091131536,410.48478140359


## Print the table row that contains the nearest object

In [9]:
star_table[star_table['distance'] == star_table['distance'].min()]

source_id,ra,dec,phot_g_mean_mag,bp_rp,parallax,distance
,deg,deg,mag,mag,mas,
int64,float64,float64,float32,float32,float64,float64
932778562099849856,119.03617666610461,47.55034822924161,9.888519,0.895524,10.587155139199597,94.45408014259073


## Print the table row that contains the bluest object

In [10]:
star_table[star_table['bp_rp'] == star_table['bp_rp'].min()]

source_id,ra,dec,phot_g_mean_mag,bp_rp,parallax,distance
,deg,deg,mag,mag,mas,
int64,float64,float64,float32,float32,float64,float64
932851782702640128,121.1837299677284,47.28846093314972,9.224431,0.59405327,3.9774703932418434,251.41607633311597


## Print the table row that contains the reddest object?

In [11]:
star_table[star_table['bp_rp'] == star_table['bp_rp'].max()]

source_id,ra,dec,phot_g_mean_mag,bp_rp,parallax,distance
,deg,deg,mag,mag,mas,
int64,float64,float64,float32,float32,float64,float64
932636037905290368,118.73226680222825,46.78772676234383,9.224494,2.5928154,0.6390501019258605,1564.8225342369403


### Due Mon Jan 22 - 1 pm
- `Save notebook as .html: File -> Download as -> HTML`
- `Upload .html to Canvas`